In [7]:
%gui qt5

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt

In [9]:
import starfish.data
import numpy as np

This notebook demonstrates how to load SeqFISH data into starfish. Below loads a fraction of fov_000, a single field of view from the Intron-SeqFISH paper. We load the test data as the full dataset is ~ 5GB and may be difficult for some laptops to load. 

In [10]:
experiment = starfish.data.SeqFISH(use_test_data=True)

ConnectionError: HTTPSConnectionPool(host='d2nhj9g34unfro.cloudfront.net', port=443): Max retries exceeded with url: /browse/formatted/20181211/seqfish-TEST/primary_image.json (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11d08a6a0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
fov = experiment['fov_000']
stack = fov['primary']

SeqFISH wants to find approximately identical numbers of spots in each round. Try a few filters to remove background. 

In [ ]:
from starfish.image import Filter

In [ ]:
glp = Filter.GaussianLowPass(sigma=1)
mhp = Filter.MeanHighPass(size=3)

In [ ]:
filtered = glp.run(stack, in_place=False)
filtered = mhp.run(filtered, in_place=False)

In [ ]:
filtered.show_stack_napari({})